In [ ]:
#check GPU
! nvidia-smi

Fri Apr 15 21:01:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    25W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# collapse
# Download and install the necessary dependencies
! pip install -q torch==1.4.0 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
! pip install transformers==3.5.0 fast-trees

! git clone -q https://github.com/microsoft/CodeXGLUE.git

# Download the CodeSearchNet Challenge dataset for the PYTHON programming language
!wget https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip 
!unzip -qq python.zip

     |████████████████████████████████| 753.4 MB 7.1 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
unzip:  cannot find or open python.zip, python.zip.zip or python.zip.ZIP.


In [ ]:
#data exploration

import json
from pprint import pprint
!gzip -d python/final/jsonl/test/python_test_0.jsonl.gz

with open('python/final/jsonl/test/python_test_0.jsonl', 'r') as f:
    sample_file = f.readlines()
print(sample_file[0])

pprint(json.loads(sample_file[0]))

gzip: python/final/jsonl/test/python_test_0.jsonl.gz: No such file or directory
{"repo": "soimort/you-get", "path": "src/you_get/extractors/youtube.py", "func_name": "YouTube.get_vid_from_url", "original_string": "def get_vid_from_url(url):\n        \"\"\"Extracts video ID from URL.\n        \"\"\"\n        return match1(url, r'youtu\\.be/([^?/]+)') or \\\n          match1(url, r'youtube\\.com/embed/([^/?]+)') or \\\n          match1(url, r'youtube\\.com/v/([^/?]+)') or \\\n          match1(url, r'youtube\\.com/watch/([^/?]+)') or \\\n          parse_query_param(url, 'v') or \\\n          parse_query_param(parse_query_param(url, 'u'), 'v')", "language": "python", "code": "def get_vid_from_url(url):\n        \"\"\"Extracts video ID from URL.\n        \"\"\"\n        return match1(url, r'youtu\\.be/([^?/]+)') or \\\n          match1(url, r'youtube\\.com/embed/([^/?]+)') or \\\n          match1(url, r'youtube\\.com/v/([^/?]+)') or \\\n          match1(url, r'youtube\\.com/watch/([^/?]+)')

In [ ]:
# data exploration
# collapse
import pandas as pd

from pathlib import Path
from typing import List, Optional

# Code from CodeSearchNetChallenge: https://github.com/github/CodeSearchNet/blob/master/notebooks/ExploreData.ipynb
def jsonl_list_to_dataframe(file_list, columns=['code', 'docstring']):
    """Load a list of jsonl.gz files into a pandas DataFrame."""
    return pd.concat([pd.read_json(f,
                                   orient='records', 
                                   compression='gzip',
                                   lines=True)[columns] 
                      for f in file_list], sort=False)

def get_dfs(path: Path) -> List[pd.DataFrame]:
    """Grabs the different data splits and converts them into dataframes"""
    dfs = []
    for split in ["train", "valid", "test"]:
        files = sorted((path/split).glob("**/*.gz"))
        df = jsonl_list_to_dataframe(files).rename(columns = {'code': 'mthd', 'docstring': 'cmt'})
        dfs.append(df)
        
    return dfs

path = Path('.')
df_trn, df_val, df_tst = get_dfs(path/"python/final/jsonl") #change to python
sample = 0.01
df_trn = df_trn.sample(frac = sample)
df_val = df_val.sample(frac = sample)
df_tst = df_tst.sample(frac = sample)

len(df_trn), len(df_val), len(df_tst)

(4122, 231, 222)

In [ ]:
# data exploration
df_trn.head()

,mthd,cmt
21033,"def is_valid_package_name(name, raise_error=Fa...",Test the validity of a package name string.\n\...
26531,"def get_core(self):\n """"""\n ...",Get an unsatisfiable core if the formula was p...
620,"def jquery_js(version=None, migrate=False):\n ...",A shortcut to render a ``script`` tag for the ...
26328,"def lStruct(self):\n """""" Return the lis...",Return the list of Struct that was used for cr...
18163,"def get_fleet(self, airline_key):\n """"""...",Get the fleet for a particular airline.\n\n ...


In [ ]:
# data cleaning
# Now, that we have the data, let's clean it! 
# First, we'll remove any non-ascii characters to simplify the problem so that the model only has to think about generating English comments.

# collapse
# From https://stackoverflow.com/a/27084708/5768407
def is_ascii(s):
    '''
    Determines if the given string contains only ascii characters

    :param s: the string to check
    :returns: whether or not the given string contains only ascii characters
    '''
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

df_trn = df_trn[df_trn['mthd'].apply(lambda x: is_ascii(x))]
df_val = df_val[df_val['mthd'].apply(lambda x: is_ascii(x))]
df_tst = df_tst[df_tst['mthd'].apply(lambda x: is_ascii(x))]

df_trn = df_trn[df_trn['cmt'].apply(lambda x: is_ascii(x))]
df_val = df_val[df_val['cmt'].apply(lambda x: is_ascii(x))]
df_tst = df_tst[df_tst['cmt'].apply(lambda x: is_ascii(x))]

len(df_trn), len(df_val), len(df_tst)

(4043, 225, 220)

In [ ]:
# data cleaning
'''
Now we'll add in the additional pairs of code snippets/inline comments.
P.S. One thing to note with adding these pairs is that the inline comments will appear twice in the datasets. 
The first in the method where the inline comment came from and the second in the target for the code snippet. 
This is only a problem for the training set since it allows for the model to cheat by simply remembering the inline comment from the example method it came from. 
However, in my testing, I found this to not be an issue and the model seems to still work well despite this problem. Just thought ya should know :).
'''
#collapse
from tqdm.auto import tqdm

def get_inline_pairs(mthd):
    '''
    Get all pairs of inline comments and corresponding code snippets

    :param mthd: the method to retrieve the pairs of comments and corresponding
    code snippets from
    :returns: all pairs of comments and corresponding code snippets
    '''
    pairs = [[]]

    comment = False
    bracket = False
    indent_lvl = -1
    lines = mthd.split("\n")
    for line in lines:
        if "//" in line and not bracket and not "://" in line:
            pairs[-1].append(line)
            if '\t' in line:
                indent_lvl = line.count('\t')
            else:
                indent_lvl = line.split("//")[0].count(' ')
            comment = True
            bracket = False
        elif comment:
            if '{' in line and not bracket:
                bracket = True
                pairs[-1].append(line)
            elif '}' in line:
                line_indent = -1
                if '\t' in line:
                    line_indent = line.count('\t')
                else:
                    line_indent = line.split("//")[0].count(' ')
                if indent_lvl == line_indent:
                    pairs[-1].append(line)
                if not bracket:
                    pairs.append([])
                    comment = False
                    bracket = False
            elif line.isspace() or line == '' and not bracket:
                pairs.append([])
                comment = False
            else:
                pairs[-1].append(line)
    
    # Convert pairs into proper format of (code snippet, inline comment) dataframe
    code_snippets   = []
    comments        = []
    for pair in pairs:
        if pair and len(pair) < 5:
            code    = []
            comment = []
            skip = False
            for line in pair:
                if "TODO" in line: break
                if "//" in line:
                    comment.append(line.replace('//', ''))
                else:
                    code.append(line)
            if len(code) > 1 and len(comment) > 0:
                        code_snippets.append('\n'.join(code))
                        comments.append('\n'.join(comment))

    pairs = pd.DataFrame(zip(code_snippets, comments), columns = ["mthd", "cmt"])
    return pairs


def add_inline(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Helper function to go through all methods in a given dataframe and add all
    pairs of inline comments and corresponding code snippets

    :param df: the dataframe to retrieve and add all pairs of inline comments
    and corresponding code snippets to
    :returns: a new dataframe with the newly added pairs of inline comments and
    corresponding code snippets
    '''
    new_df = df[df['mthd'].str.contains("//")]
    all_pairs = []
    for mthd in tqdm(new_df.mthd.values):
        pairs = get_inline_pairs(mthd)
        all_pairs.append(pairs)

    df_pairs = pd.concat([pairs for pairs in all_pairs])
    return pd.concat([df, df_pairs])

df_trn = add_inline(df_trn)
df_val = add_inline(df_val)
df_tst = add_inline(df_tst)

len(df_trn), len(df_val), len(df_tst)

  0%|          | 0/240 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

(4051, 227, 220)

In [ ]:
# data cleaning
# We'll also remove pairs where the size of the code is smaller than the comment.
# collapse
df_trn = df_trn[df_trn.apply(lambda row: len(row.mthd) > len(row.cmt), axis = 1)]
df_val = df_val[df_val.apply(lambda row: len(row.mthd) > len(row.cmt), axis = 1)]
df_tst = df_tst[df_tst.apply(lambda row: len(row.mthd) > len(row.cmt), axis = 1)]

len(df_trn), len(df_val), len(df_tst)

(4050, 227, 220)

In [ ]:
# data cleaning
# we'll remove any examples that have the special <code> tag since these also tend to 
# contain extra information that the model doesn't have a good hope of generating.

# collapse
def has_code(cmt: str) -> bool:
    '''
    Determinine if the given comment contains the HTML <code> tag

    :param cmt: the comment to check whether it contains the HTML <code> tag
    :returns: whether or not the given comment contains the HTML <code> tag
    '''
    if '<code>' in cmt: return True
    else: return False

df_trn = df_trn[~df_trn['cmt'].apply(lambda x: has_code(x))]
df_val = df_val[~df_val['cmt'].apply(lambda x: has_code(x))]
df_tst = df_tst[~df_tst['cmt'].apply(lambda x: has_code(x))]

len(df_trn), len(df_val), len(df_tst)

(4049, 227, 220)

In [ ]:
# data cleaning
# remove HTML
import re

# collapse
def clean_html(cmt: str) -> str:
    '''
    Remove any HTML tags from a given comment

    :param cmt: the comment to remove any HTML tags from
    :returns: the comment with any HTML tags removed
    '''
    result = re.sub(r"<.?span[^>]*>|<.?code[^>]*>|<.?p[^>]*>|<.?hr[^>]*>|<.?h[1-3][^>]*>|<.?a[^>]*>|<.?b[^>]*>|<.?blockquote[^>]*>|<.?del[^>]*>|<.?dd[^>]*>|<.?dl[^>]*>|<.?dt[^>]*>|<.?em[^>]*>|<.?i[^>]*>|<.?img[^>]*>|<.?kbd[^>]*>|<.?li[^>]*>|<.?ol[^>]*>|<.?pre[^>]*>|<.?s[^>]*>|<.?sup[^>]*>|<.?sub[^>]*>|<.?strong[^>]*>|<.?strike[^>]*>|<.?ul[^>]*>|<.?br[^>]*>", "", cmt)
    return result

df_trn.cmt = df_trn.cmt.apply(clean_html)
df_val.cmt = df_val.cmt.apply(clean_html)
df_tst.cmt = df_tst.cmt.apply(clean_html)

In [ ]:
# data cleaning
# We'll make everything lower case, remove extra whitespace, remove empty comments, and remove duplicates.
# collapse
df_trn = df_trn.applymap(lambda x: ' '.join(x.split()).lower())
df_val = df_val.applymap(lambda x: ' '.join(x.split()).lower())
df_tst = df_tst.applymap(lambda x: ' '.join(x.split()).lower())

df_trn = df_trn[~(df_trn['cmt'] == '')]
df_val = df_val[~(df_val['cmt'] == '')]
df_tst = df_tst[~(df_tst['cmt'] == '')]

df_trn = df_trn[~df_trn['cmt'].duplicated()]
df_val = df_val[~df_val['cmt'].duplicated()]
df_tst = df_tst[~df_tst['cmt'].duplicated()]

len(df_trn), len(df_val), len(df_tst)

(4020, 227, 219)

In [ ]:
df_trn.head()

,mthd,cmt
21033,"def is_valid_package_name(name, raise_error=fa...",test the validity of a package name string. ar...
26531,"def get_core(self): """""" get an unsatisfiable c...",get an unsatisfiable core if the formula was p...
620,"def jquery_js(version=none, migrate=false): ''...",a shortcut to render a ``script`` tag for the ...
26328,"def lstruct(self): """""" return the list of stru...",return the list of struct that was used for cr...
18163,"def get_fleet(self, airline_key): """"""get the f...",get the fleet for a particular airline. given ...


In [ ]:
# # data exploration, WHAT IS WRONG HERE???
# # collapse
# import numpy as np

# from collections import Counter
# from statistics import mean, median, stdev
# from transformers import AutoTokenizer

# def get_counter(df: pd.DataFrame, tokenizer: AutoTokenizer, col: str) -> Counter:
#     '''
#     Get the counts for each token in a given pandas dataframe column

#     :param df: the pandas dataframe to get the counts of tokens from
#     :param tokenizer: the tokenizer to use for tokenizing the rows in the pandas
#     dataframe
#     :param col: the column to grab rows from when tokenizing
#     :returns: the counts of each token in the given pandas dataframe
#     column
#     '''
#     toks = []
#     for i, row in df.iterrows():
#         toks.extend(tokenizer.tokenize(row[col]))
            
#     cnt = Counter()
#     for tok in toks:
#         cnt[tok] += 1  
#     return cnt

# tokenizer = AutoTokenizer.from_pretrained('microsoft/codebert-base')
# mthd_cnt = get_counter(df_trn, tokenizer, 'mthd')
# cmt_cnt = get_counter(df_trn, tokenizer, 'cmt')
# mthd_lens = df_trn.mthd.apply(lambda x: len(tokenizer.tokenize(x))).values
# cmt_lens = df_trn.cmt.apply(lambda x: len(tokenizer.tokenize(x))).values
# max_mthd_len = int(np.quantile(mthd_lens, 0.95))
# max_cmt_len = int(np.quantile(cmt_lens, 0.95))

In [ ]:
#TRAIN START
# output the data into the structure that the awesome CodeXGLUE Code-Text benchmark expects.
% cd CodeXGLUE/Code-Text/code-to-text/code
! ls
! mkdir python

import json

df_trn['code_tokens'] = df_trn.mthd.apply(lambda x: x.split())
df_trn['docstring_tokens'] = df_trn.cmt.apply(lambda x: x.split())
with open('python/train.jsonl','w') as f:
    for _, row in df_trn.iterrows():
        f.write(json.dumps(row.to_dict()) + '\n')

df_val['code_tokens'] = df_val.mthd.apply(lambda x: x.split())
df_val['docstring_tokens'] = df_val.cmt.apply(lambda x: x.split())
with open('python/valid.jsonl','w') as f:
    for _, row in df_val.iterrows():
        f.write(json.dumps(row.to_dict()) + '\n')

df_tst['code_tokens'] = df_tst.mthd.apply(lambda x: x.split())
df_tst['docstring_tokens'] = df_tst.cmt.apply(lambda x: x.split())
with open('python/test.jsonl','w') as f:
    for _, row in df_tst.iterrows():
        f.write(json.dumps(row.to_dict()) + '\n')

[Errno 2] No such file or directory: 'CodeXGLUE/Code-Text/code-to-text/code'
/content/CodeXGLUE/Code-Text/code-to-text/code
bleu.py  model.py  run.py


In [ ]:


lang = 'python' # programming language
lr = 5e-5
batch_size = 16 # change depending on the GPU Colab gives you
beam_size = 10
source_length = 256
target_length = 128
data_dir = '.'
output_dir = f'model/{lang}'
train_file = f'{data_dir}/{lang}/train.jsonl'
dev_file = f'{data_dir}/{lang}/valid.jsonl'
epochs = 10 
pretrained_model = 'microsoft/codebert-base'

! python run.py \
    --do_train \
    --do_eval \
    --do_lower_case \
    --model_type roberta \
    --model_name_or_path {pretrained_model} \
    --train_filename {train_file} \
    --dev_filename {dev_file} \
    --output_dir {output_dir} \
    --max_source_length {source_length} \
    --max_target_length {target_length} \
    --beam_size {beam_size} \
    --train_batch_size {batch_size} \
    --eval_batch_size {batch_size} \
    --learning_rate {lr} \
    --num_train_epochs {epochs}

04/15/2022 21:28:58 - INFO - __main__ -   Namespace(adam_epsilon=1e-08, beam_size=10, config_name='', dev_filename='./python/valid.jsonl', do_eval=True, do_lower_case=True, do_test=False, do_train=True, eval_batch_size=16, eval_steps=-1, gradient_accumulation_steps=1, learning_rate=5e-05, load_model_path=None, local_rank=-1, max_grad_norm=1.0, max_source_length=256, max_steps=-1, max_target_length=128, model_name_or_path='microsoft/codebert-base', model_type='roberta', no_cuda=False, num_train_epochs=10, output_dir='model/python', seed=42, test_filename=None, tokenizer_name='', train_batch_size=16, train_filename='./python/train.jsonl', train_steps=-1, warmup_steps=0, weight_decay=0.0)
04/15/2022 21:28:58 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False
Downloading: 100% 498/498 [00:00<00:00, 424kB/s]
Downloading: 100% 899k/899k [00:00<00:00, 5.28MB/s]
Downloading: 100% 456k/456k [00:00<00:00, 3.12MB/s]
Downloading: 100% 150/150 [00:00<00:0